# Testing accuracy of Stock Price Predictions made using XGBOOST

In [3]:
!pip install scikit-learn
!pip install pandas
!pip install plotly
import sys
!{sys.executable} -m pip install xgboost

You should consider upgrading via the 'c:\users\sahur\appdata\local\programs\python\python38\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\users\sahur\appdata\local\programs\python\python38\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\users\sahur\appdata\local\programs\python\python38\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\users\sahur\appdata\local\programs\python\python38\python.exe -m pip install --upgrade pip' command.


In [4]:
from IPython.core.debugger import set_trace

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from xgboost import XGBRegressor
import time
import yfinance as yf
plt.style.use(style="seaborn")
%matplotlib inline

In [5]:
df = pd.read_csv("data/TSLA-max-hourly.csv")
df.dtypes

Date          object
Close/Last    object
Volume         int64
Open          object
High          object
Low           object
dtype: object

### Data cleaning and Preprocessing

In [6]:
#changing data types
df['Date'] = pd.to_datetime(df['Date'])
df['Close/Last'] = df['Close/Last'].replace({'\$':''}, regex = True)
df['Close/Last'] = pd.to_numeric(df['Close/Last'])
df['Open'] = df['Open'].replace({'\$':''}, regex = True)
df['Open'] = pd.to_numeric(df['Open'])
df['High'] = df['High'].replace({'\$':''}, regex = True)
df['High'] = pd.to_numeric(df['High'])
df['Low'] = df['Low'].replace({'\$':''}, regex = True)
df['Low'] = pd.to_numeric(df['Low'])
df.dtypes

Date          datetime64[ns]
Close/Last           float64
Volume                 int64
Open                 float64
High                 float64
Low                  float64
dtype: object

In [7]:
df.head(5)

,Date,Close/Last,Volume,Open,High,Low
0,2021-07-15,650.60,20209570,658.390,666.1400,637.8800
1,2021-07-14,653.38,21641190,670.750,678.6099,652.8400
2,2021-07-13,668.54,20966090,686.320,693.2800,666.2958
3,2021-07-12,685.70,25927040,662.200,687.2400,662.1610
4,2021-07-09,656.95,18140550,653.185,658.9100,644.6900


In [8]:
df_ = df[['Close/Last']].copy()
df_.shape

(2516, 1)

In [9]:
df.head(5)

,Date,Close/Last,Volume,Open,High,Low
0,2021-07-15,650.60,20209570,658.390,666.1400,637.8800
1,2021-07-14,653.38,21641190,670.750,678.6099,652.8400
2,2021-07-13,668.54,20966090,686.320,693.2800,666.2958
3,2021-07-12,685.70,25927040,662.200,687.2400,662.1610
4,2021-07-09,656.95,18140550,653.185,658.9100,644.6900


## Transform this to a supervised learning problem.

In [10]:
df_["target"] = df_['Close/Last'].shift(-1)


In [11]:
df_.dropna(inplace=True) #drop nan values
df_.shape

(2515, 2)

In [12]:
df_.head(5)

,Close/Last,target
0,650.60,653.38
1,653.38,668.54
2,668.54,685.70
3,685.70,656.95
4,656.95,652.81


### Train Test data Split

In [13]:
def train_test_split(data, perc):
    data = data.values
    n = int(len(data) * (1 - perc))
    return data[:n], data[n:]

In [14]:
train, test = train_test_split(df_, 0.2)  #80 percent training, 20 percent testing
print(len(train))
print(len(test))

2012
503


Take the train set and test input row as input, fit a model, and make a prediction using XGBRegressor of scikit-learn.

In [15]:
X = train[:, :-1]
y = train[:, -1]
X

array([[650.6  ],
       [653.38 ],
       [668.54 ],
       ...,
       [ 24.548],
       [ 24.486],
       [ 23.936]])

In [16]:
y

array([653.38 , 668.54 , 685.7  , ...,  24.486,  23.936,  23.806])

In [17]:
model = XGBRegressor(objective="reg:squarederror", n_estimators=1000)
model.fit(X, y)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.300000012, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=1000, n_jobs=8, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [18]:
test[0]

array([23.806, 24.05 ])

In [19]:
val = np.array(test[0, 0]).reshape(1, -1)

pred = model.predict(val)
print(pred[0])

23.835068


### Predict one sample at a time

In [20]:
def xgb_predict(train, val):
    train = np.array(train)
    X, y = train[:, :-1], train[:, -1]
    model = XGBRegressor(objective="reg:squarederror", n_estimators=1000)
    model.fit(X, y)

    val = np.array(val).reshape(1, -1)
    pred = model.predict(val)
    return pred[0]

In [21]:
xgb_predict(train, test[0, 0])

23.835068

### Walk-forward validation

 Making a one step forward prediction, then adding the real observation from the test set to the train set, refit the model, then predict the next step in the test dataset.

In [23]:
from sklearn.metrics import mean_squared_error


def validate(data, perc):
    predictions = []

    train, test = train_test_split(data, perc)

    history = [x for x in train]

    for i in range(len(test)):
        test_X, test_y = test[i, :-1], test[i, -1]

        pred = xgb_predict(history, test_X[0])
        predictions.append(pred)

        history.append(test[i])

    error = mean_squared_error(test[:, -1], predictions, squared=False)

    return error, test[:, -1], predictions

In [25]:
%%time
rmse, y, pred = validate(df_, 0.2)

print(rmse)

c:\users\sahur\appdata\local\programs\python\python38\lib\site-packages\xgboost\data.py:119: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(


0.5480157218707934
Wall time: 29min 11s


### So our predicted model has low RMSE. But still it is not the most accurate model to predict stock price trend. But for long term investements, one may trust this predictions.